[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SlimeVRX/SlimeVRX/blob/main/hipnuc/document/14.ipynb)

#**Study [Attitude for IMU & AHRS](https://zhuanlan.zhihu.com/p/351596374)**
##**Lesson 14: Earth, Gravity and Coordinate Systems** [(地球，重力与坐标系)](https://zhuanlan.zhihu.com/p/360744867)

> Hệ tọa độ tuy đã được giới thiệu từ trước nhưng không chuyên sâu và khiến mọi người cảm thấy hơi hoa mắt, bài viết này sẽ ôn tập lại các khái niệm chính thức về hệ tọa độ, hệ thống trái đất và lực hấp dẫn.

### Mô hình trái đất

Hình dạng của trái đất không đều, bề mặt bị rỗ và nó không phải là hình cầu hoàn hảo. Làm thế nào chúng ta có thể thiết lập một mô hình toán học chính xác để nghiên cứu nó?

Nói chung gần đúng như một hình cầu, trục quay của trái đất (trục cực) trùng với bán trục nhỏ của hình elip ~~, và độ lớn của hình elip bằng khoảng 1/300 (sự khác biệt giữa bán trục dài và ngắn là khoảng 21 km) (Trái đất là một khối).~~ Sự quay của một hình elip quanh bán trục nhỏ của nó tạo nên bề mặt của một hình cầu và đường xích đạo của Trái đất là hình tròn trong mô tả này.

<p align="left"><img width="350", src="https://pic4.zhimg.com/80/v2-f0e7f75dd63506af5f79a28fe75415c7_720w.jpg"></p>

Một số khái niệm quan trọng:

Khi tôi không thể đọc hiểu lý thuyết được nữa -> tôi chuyển sang viết code, code dễ hiểu hơn nhiều

### Hệ tọa độ chung

### Trọng lực

In [26]:
import numpy as np

sin = np.sin
cos = np.cos
asin = np.arcsin
atan2 = np.arctan2
atan = np.arctan

In [6]:
def ch_gravity(lat, h):
    gravity = 9.780325 * ( 1 + 0.0053024*sin(lat)**2 - 0.0000058*sin(lat*2)**2)
    return gravity

In [7]:
latd = np.array([45, 45, 45, 0, 90, 60])

for i in range(len(latd)):
  gravity = ch_gravity(np.deg2rad(latd[i]), 0)
  # Vĩ độ -> Trọng lực
  print("Latitude:%f -> Gravity:%f\n" %(latd[i], gravity))

Latitude:45.000000 -> Gravity:9.806198

Latitude:45.000000 -> Gravity:9.806198

Latitude:45.000000 -> Gravity:9.806198

Latitude:0.000000 -> Gravity:9.780325

Latitude:90.000000 -> Gravity:9.832184

Latitude:60.000000 -> Gravity:9.819177



### Chuyển đổi giữa dòng LLA và ECEF

In [15]:
def ch_LLA2ECEF(lat, lon, h):
    # % 经纬高转ECEF坐标
    # % lat:纬度(rad)
    # % lon:经度(rad)
    # % h高度(m)
    R0 = 6378137          # WGS84 Equatorial radius in meters
    e = 0.0818191908425   # WGS84 eccentricity

    # Calculate transverse radius of curvature using (2.105)
    RN = R0 / np.sqrt(1 - (e * sin(lat))**2)

    # Convert position using (2.112)
    clat = cos(lat)
    slat = sin(lat)
    clon = cos(lon)
    slon = sin(lon)

    XYZ = np.array([0, 0, 0], dtype=np.float64)

    XYZ[0] = (RN + h) * clat * clon
    XYZ[1] = (RN + h) * clat * slon
    XYZ[2] = ((1 - e**2) * RN + h) * slat

    return XYZ

In [24]:
def ch_ECEF2LLA(XYZ):
    # % ECEF坐标转经纬高
    # % lat:纬度(rad)
    # % lon:经度(rad)
    # % h高度(m)

    R_0 = 6378137           # WGS84 Equatorial radius in meters
    e = 0.0818191908425     # WGS84 eccentricity

    lon = atan2(XYZ[1],XYZ[0])

    # From (C.29) and (C.30)
    k1 = np.sqrt(1 - e**2) * np.abs(XYZ[2])
    k2 = e**2 * R_0
    beta = np.sqrt(XYZ[0]**2 + XYZ[1]**2)
    E = (k1 - k2) / beta
    F = (k1 + k2) / beta

    # From (C.31)
    P = 4/3 * (E*F + 1)

    # From (C.32)
    Q = 2 * (E**2 - F**2)

    # From (C.33)
    D = P**3 + Q**2

    # From (C.34)
    V = (np.sqrt(D) - Q)**(1/3) - (np.sqrt(D) + Q)**(1/3)

    # From (C.35)
    G = 0.5 * (np.sqrt(E**2 + V) + E)

    # From (C.36)
    T = np.sqrt(G**2 + (F - V * G) / (2 * G - E)) - G

    # From (C.37)
    lat = np.sign(XYZ[2]) * atan((1 - T**2) / (2 * T * np.sqrt(1 - e**2)))

    # From (C.38)
    h = (beta - R_0 * T) * cos(lat) + (XYZ[2] - np.sign(XYZ[2]) * R_0 * np.sqrt(1 - e**2)) * sin (lat)

    return [lat, lon, h]

In [28]:
latd = 45
lond = 30
h = 1000

lat = np.deg2rad(latd)
lon = np.deg2rad(lond)

print("EX1: Latitude and Longitude: %.4f° %.4f° %.2fm\n" %(latd, lond ,h))

XYZ = ch_LLA2ECEF(lat, lon, h)
print("Convert to ECEF system coordinates: %.3f(m), %.3f(m) ,%.3f(m)\n\n" %(XYZ[0], XYZ[1], XYZ[2]))

XYZ = np.array([3912960.837, 2259148.993, 4488055.516])
print("EX2: ECEF coordinates: %.3f(m) %.3f(m) %.3f(m)\n" %(XYZ[0], XYZ[1], XYZ[2]))
[lat, lon, h] = ch_ECEF2LLA(XYZ)
print("Convert to Latitude and Longitude height (LLA):%f(rad) %f(rad) %f(m)\n" %(lat, lon, h))

EX1: Latitude and Longitude: 45.0000° 30.0000° 1000.00m

Convert to ECEF system coordinates: 3912960.837(m), 2259148.993(m) ,4488055.516(m)


EX2: ECEF coordinates: 3912960.837(m) 2259148.993(m) 4488055.516(m)

Convert to Latitude and Longitude height (LLA):0.785398(rad) 0.523599(rad) 1000.000055(m)



### Chuyển đổi giữa ENU (hệ tọa độ mặt phẳng tiếp tuyến cục bộ) và LLA

In [37]:
def ch_earth(lat, lon, h):

    # %% 根据经纬度计算地球常用参数
    # % INPUT
    # % lat: 纬度(rad)
    # % lon: 经度(rad)

    # % OUTPUT
    # % R_meridian(RM)： 南北向地球曲率半径, 子午圈曲率半径(竖着的)
    # % R_transverse(RN)：东西向地球曲率半径, 卯酉圈曲率半径(横着的)
    # % C_ECEF2ENU: ECEF到ENU转换矩阵
    # % C_ECEF2NED: ECEF到NED转换矩阵

    R0 = 6378137            # WGS84 赤道半径
    e = 0.0818191908425     # WGS84 eccentricity

    # Calculate meridian radius of curvature using (2.105)
    temp = 1 - (e * sin(lat))**2
    R_meridian = R0 * (1 - e**2) / temp**1.5

    # Calculate transverse radius of curvature using (2.105)
    R_transverse = R0 / np.sqrt(temp)

    clat = cos(lat)
    slat = sin(lat)
    clon = cos(lon)
    slon = sin(lon)

    # C_ECEF2ENU = np.array([0, 0, 0], dtype=np.float64)
    # C_ECEF2ENU[0,:] = [-slon ,             clon,                 0]
    # C_ECEF2ENU[1,:] = [ -slat*clon,    -slat*slon,          clat]
    # C_ECEF2ENU[2,:] = [ clat*clon,      clat*slon,          slat]

    C_ECEF2ENU = np.array([[-slon ,             clon,                 0],
                           [-slat*clon,    -slat*slon,          clat],
                           [ clat*clon,      clat*slon,          slat]],dtype=np.float64)
    
    # C_ECEF2NED = np.array([0, 0, 0], dtype=np.float64)
    # C_ECEF2NED[0,:] = [-slat*clon,     -slat * slon,       clat]
    # C_ECEF2NED[1,:] = [-slon,             clon,                    0]
    # C_ECEF2NED[2,:] = [ -clat*clon,   -clat*slon,        -slat]

    C_ECEF2NED = np.array([[-slat*clon,     -slat * slon,       clat],
                           [-slon,             clon,                    0],
                           [ -clat*clon,   -clat*slon,        -slat]], dtype=np.float64)

    return [R_meridian, R_transverse, C_ECEF2ENU, C_ECEF2NED]

In [48]:
def ch_LLA2ENU(lat, lon, h, lat0, lon0, h0):
    # % 经纬高 转 ENU
    # % lat0, lon0, h0: 起始点经纬高, 经纬度为rad， 高度为m
    # % lat, lon, h 终点经纬高, 经纬度为rad， 高度为m
    # % E, N ,U 系下增量，单位为m

    # 精确算法
    XYZ0 = ch_LLA2ECEF(lat0, lon0, h0)
    XYZ1 = ch_LLA2ECEF(lat, lon, h)
    dXYZ = XYZ1 - XYZ0

    [_, _, C_ECEF2ENU, _] = ch_earth(lat0, lon0, h0)
    dENU = C_ECEF2ENU.dot(dXYZ)
    E = dENU[0]
    N = dENU[1]
    U = dENU[2]
    
    # 近似算法
    # % R_0 = 6378137; %WGS84 Equatorial radius in meters
    # % clat = cos(lat0);
    # % E = (lon - lon0) * clat * R_0;
    # % N =  (lat - lat0) * R_0;
    # % U = h - h0;

    return [E, N, U] 

In [55]:
def ch_ENU2LLA(E, N ,U, lat0, lon0, h0):

    # % ENU 转 经纬高
    # % E, N ,U 系下增量，单位为m
    # % lat0, lon0, h0: 起始点经纬高, 经纬度为rad， 高度为m
    # % lat, lon, h 终点经纬高, 经纬度为rad， 高度为m

    # 精确解
    XYZ0 = ch_LLA2ECEF(lat0, lon0, h0)
    [_, _, C_ECEF2ENU, _]= ch_earth(lat0, lon0, h0)
    dXYZ = C_ECEF2ENU.T.dot(np.array([E, N, U], dtype=np.float64).T)
    XYZ = dXYZ + XYZ0
    [lat, lon, h] = ch_ECEF2LLA(XYZ)

    return [lat, lon, h]

In [58]:
lat0 = np.deg2rad(46.017)
lon0 = np.deg2rad(7.750)
h0 = 1673

lat = np.deg2rad(45.976)
lon = np.deg2rad(7.658)
h = 4531

[E, N, U] = ch_LLA2ENU(lat, lon, h, lat0, lon0, h0)

print("EX1 - origin LLA: %f %f %f, target LLA: %f %f %f \n" %(lat0, lon0, h0, lat, lon, h))
print("ENU increment after conversion: %f %f %f\n" %(E, N, U))

lat0 = np.deg2rad(46.017)
lon0 = np.deg2rad(7.750)
h0 = 1673

xEast = -7134.8
yNorth = -4556.3
zUp = 2852.4

[lat, lon, h] = ch_ENU2LLA(xEast, yNorth, zUp, lat0, lon0, h0)
print("EX2 - origin LLA: %f %f %f, ENU system down increment: %f %f %f \n" %(lat0, lon0, h0, xEast, yNorth, zUp))
print("Convert to get the target point LLA:%f(deg) %f(deg) %f(m)\n" %(np.rad2deg(lat), np.rad2deg(lon), h))


EX1 - origin LLA: 0.803148 0.135263 1673.000000, target LLA: 0.802433 0.133657 4531.000000 

ENU increment after conversion: -7134.757196 -4556.321514 2852.390424

EX2 - origin LLA: 0.803148 0.135263 1673.000000, ENU system down increment: -7134.800000 -4556.300000 2852.400000 

Convert to get the target point LLA:45.976000(deg) 7.657999(deg) 4531.009608(m)



### Chuyển đổi giữa ECEF và ENU

In [59]:
def ch_ECEF2ENU(XYZ, lat0, lon0, h0):

  # % ECEF坐标转ENU
  # % lat0, lon0, h0: 起始点经纬高, 经纬度为rad， 高度为m
  # % XYZ ECEF坐标  单位为m
  # % ENU距离 单位为m

  [lat, lon, h] = ch_ECEF2LLA(XYZ)

  [E, N, U] =  ch_LLA2ENU(lat, lon, h,  lat0, lon0, h0)

  return [E, N, U]

In [60]:
def ch_ENU2ECEF(E, N, U, lat0, lon0, h0):
  # % ENU  转 ECEF
  # % lat0, lon0, h0: 起始点经纬高, 经纬度为rad， 高度为m
  # % ENU东北天距离，单位为m
  # % 返回 ECEF坐标 单位m

  [lat, lon, h] = ch_ENU2LLA(E, N, U, lat0, lon0, h0)
  
  XYZ = ch_LLA2ECEF(lat, lon, h)

  return XYZ

In [62]:
lat0 = np.deg2rad(45.9132)
lon0 = np.deg2rad(36.7484)
h0 = 1877753.2

x = 5507528.9
y = 4556224.1
z = 6012820.8

[E, N, U] = ch_ECEF2ENU([x, y, z], lat0, lon0, h0)
print("EX1 - Knowing ECEF coordinates of LLA, find in the ENU system\n")
print("ENU offset: %f %f %f\n" %(E, N, U))

E = 355601.261550
N = -923083.155899
U = 1041016.423793

XYZ = ch_ENU2ECEF(E, N, U, lat0, lon0, h0)
print("EX2 - Knowing LLA and ENU, find the ECEF coordinates\n")
print("ECEF coordinates: %f %f %f\n" %(XYZ[0], XYZ[1], XYZ[2]))

EX1 - Knowing ECEF coordinates of LLA, find in the ENU system

ENU offset: 355601.261550 -923083.155899 1041016.423793

EX2 - Knowing LLA and ENU, find the ECEF coordinates

ECEF coordinates: 5507528.900000 4556224.100000 6012820.800000

